# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [1]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '192.168.84.193',
 'raylet_ip_address': '192.168.84.193',
 'redis_address': '192.168.84.193:42333',
 'object_store_address': '/tmp/ray/session_2021-12-27_19-09-37_270787_32382/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-27_19-09-37_270787_32382/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-12-27_19-09-37_270787_32382',
 'metrics_export_port': 60353,
 'node_id': 'b40a081a277157810460eece7e0a2c112c5b99dbd3e6fe66da31cb64'}

## Remote Classes example

To start, we'll define a class and use the decorator:

In [2]:
@ray.remote
class Counter:
    def __init__ (self):
        self.value = 0

    def increment (self):
        self.value += 1
        return self.value

Now use this class `Counter` to create an actor:

In [3]:
%%time

counter = Counter.remote()

CPU times: user 9.59 ms, sys: 285 µs, total: 9.88 ms
Wall time: 14 ms


Then call the actor:

In [4]:
%%time

obj_ref = counter.increment.remote()
ray.get(obj_ref)

CPU times: user 5.78 ms, sys: 0 ns, total: 5.78 ms
Wall time: 5.36 ms


1

Use list comprehension to show the state being maintained in the actor

In [5]:
%%time

f_list = [counter.increment.remote() for _ in range(3)]

CPU times: user 0 ns, sys: 571 µs, total: 571 µs
Wall time: 422 µs


In [6]:
%%time 

print(ray.get(f_list))

[2, 3, 4]
CPU times: user 719 µs, sys: 142 µs, total: 861 µs
Wall time: 608 µs


In [7]:
f_list

[ObjectRef(69a6825d641b46137e513a1de80c0fa47a0505b10100000001000000),
 ObjectRef(ee4e90da584ab0eb7e513a1de80c0fa47a0505b10100000001000000),
 ObjectRef(4ee449587774c1f07e513a1de80c0fa47a0505b10100000001000000)]


Let's use another Actor class and create multiple instances associated with a distinct attribute, such as a name.

In [9]:
from random import randint

@ray.remote
class GoalsScored:
    def __init__ (self, player) -> None:
        self._goals = 0
        self._player = player

    def score (self) -> object:
        self._goals += randint(1, 5)
        return self._goals
       
    def player(self) -> str:
        return self._player

Define three Actors: Rolando, Neymar, Messi

In [10]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo")
neymar = GoalsScored.remote("Neymar")
messi = GoalsScored.remote("Messi")

CPU times: user 17.4 ms, sys: 7.86 ms, total: 25.3 ms
Wall time: 22.7 ms


In [11]:
%%time

ronaldo_ref = ronaldo.score.remote()
neymar_ref = neymar.score.remote()
messi_ref  = messi.score.remote()

CPU times: user 805 µs, sys: 175 µs, total: 980 µs
Wall time: 613 µs



Again, use list comprehension to iterate over each Actor instances, along with object_ref
for their scores, maintained by each distincgive actor.

In [12]:
for ref, ref_obj in [(ronaldo, ronaldo_ref), (neymar, neymar_ref), (messi, messi_ref) ]:
   print(f"Player: {ray.get(ref.player.remote())} and goals scored: {ray.get(ref_obj)}")

Player: Ronaldo and goals scored: 1
Player: Neymar and goals scored: 1
Player: Messi and goals scored: 1


## queuing function call

In [2]:
import time
@ray.remote
class SleepCounter:
    def __init__ (self):
        self.value = 0

    def sleep_increment(self):
        time.sleep(2)
        self.value += 1
        return self.value

In [3]:
%%time

counter = SleepCounter.remote()

CPU times: user 4.56 ms, sys: 4.13 ms, total: 8.68 ms
Wall time: 8.14 ms


In [6]:
%%time
f_list = [counter.sleep_increment.remote() for _ in range(3)]
print(ray.get(f_list))

[4, 5, 6]
CPU times: user 330 ms, sys: 75.7 ms, total: 405 ms
Wall time: 6.02 s


In [7]:
%%time
f_list = [counter.sleep_increment.remote() for _ in range(5)]
print(ray.get(f_list))

[7, 8, 9, 10, 11]
CPU times: user 516 ms, sys: 123 ms, total: 639 ms
Wall time: 10 s


Finally, shutdown Ray

In [ ]:
ray.shutdown()

---
## References

["A Universal Modular Actor Formalism for Artificial Intelligence"](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  
*IJCAI* (1973)